In [2]:
import json

import numpy as np

In [26]:
def get_min_max_candidate_pairs(in_predictions):
    min_distance, max_distance = np.inf, -np.inf
    min_distance_pred, max_distance_pred = None, None

    for pred in in_predictions:
        scores = list(map(lambda x: x['score'], pred['candidates']))
        if not len(scores):
            continue
        min_score, max_score = min(scores), max(scores)
        if max_score - min_score < min_distance:
            min_distance = max_score - min_score
            min_distance_pred = pred
        if max_distance < max_score - min_score:
            max_distance = max_score - min_score
            max_distance_pred = pred
    return {'min': {'distance': min_distance, 'prediction': min_distance_pred},
            'max': {'distance': max_distance, 'prediction': max_distance_pred}}

In [15]:
def print_prediction(in_prediction):
    print('ID: ', in_prediction['id'])
    print('INPUT:')
    for turn in in_prediction['input']:
        print('{}: {}'.format(turn['side'], turn['utterance']))
    print('TARGET:')
    print(in_prediction['target'])
    print('CANDIDATES:')
    for cand_turn in in_prediction['candidates']:
        print('{} [{}] [{}]'.format(cand_turn['utterance'], cand_turn['score'], cand_turn['method']))

In [4]:
LOG_FILES = ['metalwoz_pure_ext_info_parsed.json', 'metalwoz_pure_ext_info_parsed.json', 'multiwoz_ext_info_parsed.json']

In [5]:
logs = []
for log_file in LOG_FILES:
    with open(log_file) as log_in:
        logs.append(json.load(log_in))

# MetaLWOz pure task

In [8]:
min_max_preds = get_min_max_candidate_pairs(logs[0])

### Prediction with the closest retrieved and generated scores

In [16]:
min_pred = min_max_preds['min']
print_prediction(min_pred['prediction'])

ID:  0dd81289
INPUT:
Wizard: Hello how may I help you?
User: Hello, can you help me with my reservation?
Wizard: I sure can. Can you please give me a reservation number?
User: The number is 572839
Wizard: Thanks! What do you need help with?
User: I want to cancel my reservation in New York.
Wizard: Unfortunately, I can not cancel reservations. You need to contact the hotel directly. I can give you the
User: can you give me the number?
Wizard: Sure. The number io 888-888-8888.
TARGET:
Okay I'll call them right now and
CANDIDATES:
thank you [2.483] [retrieved]
thank you [2.483] [generated]


### Prediction with the most distant retrieved and generated scores

In [17]:
max_pred = min_max_preds['max']
print_prediction(max_pred['prediction'])

ID:  c9595979
INPUT:
Wizard: Hello how may I help you?
User: I need some tourism help.
Wizard: where would you like to travel to?
User: Montreal. I''m going there this summer. Where are some good spots to go?
Wizard: it depends on how much your are willing ti spent.
User: Well, I'm looking for some festivals in the area around that time. Can you just give me some suggestions?
Wizard: what is your budget?
TARGET:
Budget doesn't matter. I just want to know where I can go.
CANDIDATES:
My budget is $15.00 per ticket. [2.109] [generated]
Great! Thanks so muvh [-14.034] [retrieved]


# MetaLWOz cross task

In [18]:
min_max_preds = get_min_max_candidate_pairs(logs[1])

### Prediction with the closest retrieved and generated scores

In [19]:
min_pred = min_max_preds['min']
print_prediction(min_pred['prediction'])

ID:  0dd81289
INPUT:
Wizard: Hello how may I help you?
User: Hello, can you help me with my reservation?
Wizard: I sure can. Can you please give me a reservation number?
User: The number is 572839
Wizard: Thanks! What do you need help with?
User: I want to cancel my reservation in New York.
Wizard: Unfortunately, I can not cancel reservations. You need to contact the hotel directly. I can give you the
User: can you give me the number?
Wizard: Sure. The number io 888-888-8888.
TARGET:
Okay I'll call them right now and
CANDIDATES:
thank you [2.483] [retrieved]
thank you [2.483] [generated]


### Prediction with the most distant retrieved and generated scores

In [20]:
max_pred = min_max_preds['max']
print_prediction(max_pred['prediction'])

ID:  c9595979
INPUT:
Wizard: Hello how may I help you?
User: I need some tourism help.
Wizard: where would you like to travel to?
User: Montreal. I''m going there this summer. Where are some good spots to go?
Wizard: it depends on how much your are willing ti spent.
User: Well, I'm looking for some festivals in the area around that time. Can you just give me some suggestions?
Wizard: what is your budget?
TARGET:
Budget doesn't matter. I just want to know where I can go.
CANDIDATES:
My budget is $15.00 per ticket. [2.109] [generated]
Great! Thanks so muvh [-14.034] [retrieved]


# MultiWoz

In [27]:
min_max_preds = get_min_max_candidate_pairs(logs[2])

### Prediction with the closest retrieved and generated scores

In [28]:
min_pred = min_max_preds['min']
print_prediction(min_pred['prediction'])

ID:  SNG0284
INPUT:
Wizard: Hello, how may I help you?
User: I want a train from london liverpool street arriving at 9:30
Wizard: Which day will you be traveling?
User: I want to leave on Thursday.
Wizard: Ok, I should be able to do that for you.
User: I want to go to Cambridge, how much will that cost?
Wizard: There are three trains that will fit your needs, and the tickets are 16.60 pounds each. Would you like me to
User: Yes, that sounds good. Thank you.
Wizard: Would you like me to book that for you?
User: Can you tell me how much the ticket is?
Wizard: Yes, there are 3 that cost 16.60 pounds. You could leave at 5:39 if you 'd like. Can I book that for you?
TARGET:
No, thanks. No need. I will do that later. Thank you for all the info. Have a nice day.
CANDIDATES:
Yes, can you book it for 5 people and please give me the reference number. [2.755] [retrieved]
No, that's fine. I 'll be sure to go ahead and book that for now [2.755] [generated]


### Prediction with the most distant retrieved and generated scores

In [29]:
max_pred = min_max_preds['max']
print_prediction(max_pred['prediction'])

ID:  SNG1313
INPUT:
Wizard: Hello, how may I help you?
User: i am looking for a train. The train should leave on Monday and should arrive by 12:45.
Wizard: Where would you like to go?
TARGET:
I would like to depart from Cambridge, and travel to Bishops Stortford.
CANDIDATES:
i would like to go to Cambridge. [5.253] [generated]
I would like to leave after 09:00 on Sunday. [-11.019] [retrieved]
